<a href="https://colab.research.google.com/github/nonotoy/yubimoji/blob/main/yubimoji_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization, Dropout, GlobalAveragePooling1D, Flatten

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

dataset = '/content/drive/MyDrive/Colab Notebooks/JSL/point_history_normalised.csv'

model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/LSTM/gesture_classifier.hdf5'
tflite_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/LSTM/keypoint_classifier.tflite'

In [15]:
# モデルのパラメータ
n_features = 40  # ランドマークの数
n_sequence = 30  # シーケンス長 (調整が必要)
n_classes = 76 # 出力クラス (指文字) の数

# 2列目以降を学習データとする
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, n_features+1))) # (20730, 40)

# 1列目を正解ラベルとする
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

# ラベルをワンホットエンコードする
y_dataset = to_categorical(y_dataset, num_classes=n_classes) # (20730, 76)

In [16]:
print(X_dataset.shape[0] // n_sequence)  # 691

691


In [17]:
# サンプル数を n_sequence で割り切れるように調整
total_samples = len(X_dataset)
max_samples = total_samples - total_samples % n_sequence # 20,730 = 20,730 - 0
print(max_samples)

20730


In [18]:
X_dataset = X_dataset[:max_samples]
y_dataset = y_dataset[:max_samples]
X_dataset = X_dataset.reshape(-1, n_sequence, n_features) # (691, 30, 40)
y_dataset = y_dataset.reshape(-1, n_sequence, n_classes) # (691, 30, 76)

In [19]:
print(X_dataset.shape)
print(y_dataset.shape)

(691, 30, 40)
(691, 30, 76)


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

print(y_train.shape)
print(y_test.shape)

In [46]:
y_train_last = y_train[:, -1, :]
y_test_last = y_test[:, -1, :]

print(y_train_last.shape)
print(y_test_last.shape)

(518, 76)
(173, 76)


### LSTMモデル

In [47]:
# モデルの定義
model = Sequential([
    LSTM(128, input_shape=(n_sequence, n_features), return_sequences=False),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(n_classes, activation='softmax')
])

# モデルのコンパイル
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

# モデルのサマリー表示
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 128)               86528     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 76)                4940      
                                                                 
Total params: 99724 (389.55 KB)
Trainable params: 99724 (389.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
model.fit(
    X_train,
    y_train_last,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test_last),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
1/5 [=====>........................] - ETA: 7s - loss: 4.3778 - accuracy: 0.0312
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/LSTM/gesture_classifier.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - 3s 343ms/step - loss: 4.3525 - accuracy: 0.0154 - val_loss: 4.2737 - val_accuracy: 0.0520
Epoch 2/1000
1/5 [=====>........................] - ETA: 0s - loss: 4.2178 - accuracy: 0.0703
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/LSTM/gesture_classifier.hdf5
5/5 [==============================] - 0s 20ms/step - loss: 4.1946 - accuracy: 0.0560 - val_loss: 4.2048 - val_accuracy: 0.0694
Epoch 3/1000
1/5 [=====>........................] - ETA: 0s - loss: 4.1242 - accuracy: 0.0859
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/LSTM/gesture_classifier.hdf5
5/5 [==============================] - 0s 20ms/step - loss: 4.1193 - accuracy: 0.0772 - val_loss: 4.1458 - val_accuracy: 0.0867
Epoch 4/1000
1/5 [=====>........................] - ETA: 0s - loss: 3.9811 - accuracy: 0.1328
Epoch 4: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/LSTM/gesture_classifier.hdf5
5/5 [===================

In [49]:
input_dtype = model.input.dtype
print("Input data type:", input_dtype)

Input data type: <dtype: 'float32'>


In [50]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))

# 推論モデルとして保存
model.save(model_save_path, include_optimizer=False)

1/1 [==============================] - 0s 401ms/step


### 量子化

In [51]:
# 推論モデルを読み込み
import tensorflow as tf
from tensorflow import keras

#model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/normal/gesture_classifier.hdf5'
#tflite_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/normal/keypoint_classifier.tflite'

model = tf.keras.models.load_model(model_save_path)

In [52]:
# モデルを変換(量子化)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# TensorFlow Liteでサポートされていない操作を含むモデルに対応するための設定
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,tf.lite.OpsSet.SELECT_TF_OPS]

converter._experimental_lower_tensor_list_ops = False

tflite_quantized_model = converter.convert()
open(tflite_save_path, 'wb').write(tflite_quantized_model)

113872

In [53]:
# 推論テスト
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 推論実施の前に、入力データを適切な形状にリシェイプ
input_data = np.array([X_dataset[0]]) #.reshape(1, TIME_STEPS, DIMENSION)

# 推論実施
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[4.58011200e-04 2.28388933e-03 2.18658228e-04 3.47645604e-03
 1.16671401e-03 4.60269512e-04 5.35358023e-03 6.92924368e-06
 2.60464791e-02 1.01229220e-04 8.05379706e-04 1.54490714e-04
 2.45355128e-04 1.28673052e-03 1.26333471e-04 1.11885004e-06
 1.18379677e-02 5.55776954e-01 1.64030101e-02 6.18561025e-05
 1.04587860e-02 2.57740048e-05 1.00000550e-04 3.49430164e-04
 6.86529484e-06 1.05260779e-05 3.59549049e-05 3.59084370e-05
 8.33748054e-05 5.70532226e-04 2.37604370e-03 9.18250080e-05
 1.49548441e-06 6.00326397e-02 1.77422971e-05 6.54513075e-04
 6.13856828e-04 2.58867158e-06 1.91050513e-05 9.06567311e-06
 1.99496444e-05 6.49983167e-06 1.54487782e-06 3.95983690e-03
 4.76328743e-04 1.10206774e-05 5.33467683e-04 7.31285487e-04
 1.15637602e-06 1.97615870e-03 1.25695160e-03 4.25126730e-03
 4.06838881e-05 3.27729562e-04 2.67975440e-04 5.10538048e-05
 4.31568924e-06 1.59999039e-02 1.21966064e-01 1.22506637e-02
 9.77465606e-05 3.15506732e-05 2.17944871e-05 1.53733745e-05
 4.52529730e-06 4.472919

### 入力テンソルの形状確認

In [54]:
# モデルのパス
tflite_model_path = tflite_save_path

# TensorFlow Lite インタープリタの初期化
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# 入力ディテールの取得
input_details = interpreter.get_input_details()
print(input_details[0]['shape'])  # 入力テンソルの形状を表示

[ 1 30 40]
